## Scraping Property Access PH through Selenium
This notebook was prepared by Adem Inovejas, Christopher Lim, Czarina Tiu, and Uriel Grace Magtibay (students of DATA102 S11 Y2022-2023).  

In your notebook, make sure you have the following details outlined:
- The website scraped
- Date and time when the data was collected
- What were the challenges encountered? You may narrate or illustrate this in the notebook.
- Do you think the collected data contains any personally identifiable information (PII)?
- Conclude with your key learnings and findings.

## Importing Libaries

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import datetime
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## Setup

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
url = "https://propertyaccess.ph/offer/sale"
driver.get(url)

## Data Collection

### Collecting  the URLs of each properties

In [ ]:
has_pages = True
pages = 1
urls = []
total_time = 0
while(has_pages):  
    start = time.perf_counter()
    
    products_container = driver.find_element(by="xpath", value='//div[@class="list-product page-content"]')
    
    # gets the list of products
    items = products_container.find_elements(by="xpath", value='.//div[@class="product-wrapper"]')
    for item in items:
        # get the href of the anchor of "product-img"
        url = item.find_element(by="xpath", value='.//div[@class="product-img"]/a').get_attribute('href')
        date_published = item.find_element(by="xpath", value='.//div[@class="date"]').text
        urls.append((url, date_published))
    
    end = time.perf_counter()
    
    total_time += end - start
    print('Extracted Page %d | %d items found | %.2fs' % (pages, len(items), end - start))
    try:
        # checks if next page button is available by getting the xpath
        next_page = driver.find_element("xpath", value='//ul[@class="pargination"]/li[@class="next flex-end"]/a').get_attribute("href");
        driver.get(next_page)
        pages += 1
    except:
        # if next page button is not available, stop the loop
        has_pages = False

print('Number of pages:', pages)
print('Number of products:', len(urls))
print('Average Time per page:', total_time / len(urls))
print('Total Time:', total_time)

Extracted Page 1 | 20 items found | 0.57s


Exporting the URLs through pandas dataframe (as .csv)

In [ ]:
df = pd.DataFrame(urls, columns=['URL', 'Date Published'])
df.head()

In [ ]:
df.to_csv('property_urls.csv', index=False)

### Going through each property page to get the details

In [ ]:
df = pd.read_csv('property_urls.csv')
df.head()

In [ ]:
data = []
total_time = 0
for i, item in enumerate(list(zip(df['URL'], df['Date Published']))):
    try:
        start = time.perf_counter()
        url, date_published = item
        driver.get(url)

        # get the container which contains the property details
        property_details = driver.find_element(by="xpath", value='//div[@class="product-detail-wraper"]')

        # find the name
        name = property_details.find_element(by="xpath", value='.//h1[@class="basic-info__name"]').text
        address = property_details.find_element(by="xpath", value='.//div[@class="basic-info__street"]').text
        price = property_details.find_element(by="xpath", value='.//div[@class="basic-info__price"]').text
        author = property_details.find_element(by="xpath", value='.//div[@class="agent-name line-clamp lc-2"]').text

        # get the bedrooms, showers, parking, furnish type, total developed and lot area through utility info
        utility_info = property_details.find_element(by="xpath", value='.//div[@class="basic-info__utilities"]')

        try:
            bedrooms = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/f24d0f0d48276caeca2060fd60abf269.svg"]/..').text
        except:
            bedrooms = None

        try:
            showers = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/a6e500c94a1eaacdd59fde045fba89f0.svg"]/..').text
        except:
            showers = None

        try:
            furnish = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/9fb04d2eb34997da34e3dcfd140f51b6.svg"]/..').text
        except:
            furnish = None

        try:
            parking = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/33d487e03cfed78b821458ab3c39b574.svg"]/..').text
        except:
            parking = None

        try:
            total_developed = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/57cebe2c8bcb0aa7f604503ddb740621.svg"]/..').text
        except:
            total_developed = None

        try:
            lot_area = utility_info.find_element(by="xpath", value='//img[@data-src="https://cdn.propertyaccess.ph/prod/v1.2.5-fis-hotfix-2/da2ffe9d4238be9a3d697b4195b64fe7.svg"]/..').text
        except:
            lot_area = None

        # if section title is Property Features or Amenities
        features = [feature.text for feature in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Property Features" or text()="Amenities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Facilities
        facilities = [facility.text for facility in property_details.find_elements(by="xpath", value='.//section[@class="product-feature"]//div[text()="Facilities"]/..//div[@class="item-listing readmore-target"]//p')]

        # if section title is Nearby Places
        nearby_places = [place.text for place in property_details.find_elements(by="xpath", value='.//section[@class="product-nearby"]//div[@class="item-title"]')]

        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        end = time.perf_counter()

        total_time += end - start
        
        print('[%4d] %s | %.2fs' % (i, name, end - start))

        data.append((name, address, author, price, bedrooms, showers, parking, furnish, total_developed, lot_area, features, facilities, nearby_places, url, timestamp))
    except:
        # if page is un-available
        print('Error occured on', url)

In [ ]:
column_names = ['Name', 'Address', 'Author', 'Price', 'Bedrooms', 'Showers', 'Parking', 'Furnish', 'Total Developed', 'Lot Area', 'Features', 'Facilities', 'Nearby Places', 'URL', 'Timestamp']
df = pd.DataFrame(data, columns = column_names)
df.head()

In [ ]:
df.describe()

In [ ]:
df.to_json('unclean.json', orient='records', indent=2)

In [17]:
driver.close()

## Data Cleaning

## Challenges
- Having to go to each page in order to get the necessary data (takes longer time)
- 

## Conclusion
- [Does it have personal identifiable information?]
- 